In [1]:
# For nanogpt to transformer lens conversion
import torch
import einops
import os
import transformer_lens.utils as utils
from transformer_lens import (
    HookedTransformer,
    HookedTransformerConfig,
)
# Our pytorch model is in the nanogpt format. For easy linear probing of the residual stream, we want to convert
# it to the transformer lens format. This is done in the following code block.
# This code was developed using Neel Nanda's othello_reference/Othello_GPT.ipynb as a reference.



def convert_nanogpt_weights(old_state_dict, cfg: HookedTransformerConfig, bias: bool = False):
    """For https://github.com/karpathy/nanoGPT
    There are two complications with converting nanogpt models:
    The first is that some state dicts have an unwanted prefix on keys that needs to be removed.
    The second is that the models can be saved with or without bias. By default, there
    is no bias. This function can handle both cases."""
    # Nanogpt models saved after torch.compile() have this unwanted prefix
    # This is a simple way to remove it
    unwanted_prefix = "_orig_mod."
    for k, v in list(old_state_dict.items()):
        if k.startswith(unwanted_prefix):
            old_state_dict[k[len(unwanted_prefix) :]] = old_state_dict.pop(k)

    new_state_dict = {}
    new_state_dict["pos_embed.W_pos"] = old_state_dict["transformer.wpe.weight"]
    new_state_dict["embed.W_E"] = old_state_dict["transformer.wte.weight"]

    new_state_dict["ln_final.w"] = old_state_dict["transformer.ln_f.weight"]
    new_state_dict["ln_final.b"] = torch.zeros_like(old_state_dict["transformer.ln_f.weight"])
    new_state_dict["unembed.W_U"] = old_state_dict["lm_head.weight"].T

    if bias:
        new_state_dict["ln_final.b"] = old_state_dict["transformer.ln_f.bias"]

    for layer in range(cfg.n_layers):
        layer_key = f"transformer.h.{layer}"

        new_state_dict[f"blocks.{layer}.ln1.w"] = old_state_dict[f"{layer_key}.ln_1.weight"]
        # A bias of zeros is required for folding layer norm
        new_state_dict[f"blocks.{layer}.ln1.b"] = torch.zeros_like(
            old_state_dict[f"{layer_key}.ln_1.weight"]
        )
        new_state_dict[f"blocks.{layer}.ln2.w"] = old_state_dict[f"{layer_key}.ln_2.weight"]
        new_state_dict[f"blocks.{layer}.ln2.b"] = torch.zeros_like(
            old_state_dict[f"{layer_key}.ln_2.weight"]
        )

        W = old_state_dict[f"{layer_key}.attn.c_attn.weight"]
        W_Q, W_K, W_V = torch.tensor_split(W, 3, dim=0)
        W_Q = einops.rearrange(W_Q, "(i h) m->i m h", i=cfg.n_heads)
        W_K = einops.rearrange(W_K, "(i h) m->i m h", i=cfg.n_heads)
        W_V = einops.rearrange(W_V, "(i h) m->i m h", i=cfg.n_heads)
        new_state_dict[f"blocks.{layer}.attn.W_Q"] = W_Q
        new_state_dict[f"blocks.{layer}.attn.W_K"] = W_K
        new_state_dict[f"blocks.{layer}.attn.W_V"] = W_V

        W_O = old_state_dict[f"{layer_key}.attn.c_proj.weight"]
        W_O = einops.rearrange(W_O, "m (i h)->i h m", i=cfg.n_heads)
        new_state_dict[f"blocks.{layer}.attn.W_O"] = W_O

        new_state_dict[f"blocks.{layer}.mlp.W_in"] = old_state_dict[
            f"{layer_key}.mlp.c_fc.weight"
        ].T
        new_state_dict[f"blocks.{layer}.mlp.W_out"] = old_state_dict[
            f"{layer_key}.mlp.c_proj.weight"
        ].T

        if bias:
            new_state_dict[f"blocks.{layer}.ln1.b"] = old_state_dict[f"{layer_key}.ln_1.bias"]
            new_state_dict[f"blocks.{layer}.ln2.b"] = old_state_dict[f"{layer_key}.ln_2.bias"]
            new_state_dict[f"blocks.{layer}.mlp.b_in"] = old_state_dict[
                f"{layer_key}.mlp.c_fc.bias"
            ]
            new_state_dict[f"blocks.{layer}.mlp.b_out"] = old_state_dict[
                f"{layer_key}.mlp.c_proj.bias"
            ]

            B = old_state_dict[f"{layer_key}.attn.c_attn.bias"]
            B_Q, B_K, B_V = torch.tensor_split(B, 3, dim=0)
            B_Q = einops.rearrange(B_Q, "(i h)->i h", i=cfg.n_heads)
            B_K = einops.rearrange(B_K, "(i h)->i h", i=cfg.n_heads)
            B_V = einops.rearrange(B_V, "(i h)->i h", i=cfg.n_heads)
            new_state_dict[f"blocks.{layer}.attn.b_Q"] = B_Q
            new_state_dict[f"blocks.{layer}.attn.b_K"] = B_K
            new_state_dict[f"blocks.{layer}.attn.b_V"] = B_V
            new_state_dict[f"blocks.{layer}.attn.b_O"] = old_state_dict[
                f"{layer_key}.attn.c_proj.bias"
            ]

    return new_state_dict

In [4]:
torch.set_grad_enabled(False)
device = "cpu"
MODEL_DIR = "models/"
n_heads = 8
n_layers = 8
d_model = 512

model_name = f"lichess_{n_layers}layers_ckpt_no_optimizer.pt"
assert str(n_layers) in model_name

if not os.path.exists(f"{MODEL_DIR}{model_name}"):
    state_dict = utils.download_file_from_hf("adamkarvonen/chess_llms", model_name)
    model = torch.load(state_dict, map_location=device)
    torch.save(model, f"{MODEL_DIR}{model_name}")

checkpoint = torch.load(f"{MODEL_DIR}{model_name}", map_location=device)



In [5]:
# Print the keys of the checkpoint dictionary
model_state = checkpoint["model"]
print(len(model_state))
for key, value in model_state.items():
    print(key, value.shape)


52
_orig_mod.transformer.wte.weight torch.Size([32, 512])
_orig_mod.transformer.wpe.weight torch.Size([1023, 512])
_orig_mod.transformer.h.0.ln_1.weight torch.Size([512])
_orig_mod.transformer.h.0.attn.c_attn.weight torch.Size([1536, 512])
_orig_mod.transformer.h.0.attn.c_proj.weight torch.Size([512, 512])
_orig_mod.transformer.h.0.ln_2.weight torch.Size([512])
_orig_mod.transformer.h.0.mlp.c_fc.weight torch.Size([2048, 512])
_orig_mod.transformer.h.0.mlp.c_proj.weight torch.Size([512, 2048])
_orig_mod.transformer.h.1.ln_1.weight torch.Size([512])
_orig_mod.transformer.h.1.attn.c_attn.weight torch.Size([1536, 512])
_orig_mod.transformer.h.1.attn.c_proj.weight torch.Size([512, 512])
_orig_mod.transformer.h.1.ln_2.weight torch.Size([512])
_orig_mod.transformer.h.1.mlp.c_fc.weight torch.Size([2048, 512])
_orig_mod.transformer.h.1.mlp.c_proj.weight torch.Size([512, 2048])
_orig_mod.transformer.h.2.ln_1.weight torch.Size([512])
_orig_mod.transformer.h.2.attn.c_attn.weight torch.Size([1536, 

In [6]:

LOAD_AND_CONVERT_CHECKPOINT = True

if LOAD_AND_CONVERT_CHECKPOINT:
    synthetic_checkpoint = model_state
    for name, param in synthetic_checkpoint.items():
        if name.startswith("_orig_mod.transformer.h.0") or not name.startswith(
            "_orig_mod.transformer.h"
        ):
            print(name, param.shape)

    cfg = HookedTransformerConfig(
        n_layers=n_layers,
        d_model=d_model,
        d_head=int(d_model / n_heads),
        n_heads=n_heads,
        d_mlp=d_model * 4,
        d_vocab=32,
        n_ctx=1023,
        act_fn="gelu",
        normalization_type="LNPre",
    )
    model = HookedTransformer(cfg)
    model.to(device)

_orig_mod.transformer.wte.weight torch.Size([32, 512])
_orig_mod.transformer.wpe.weight torch.Size([1023, 512])
_orig_mod.transformer.h.0.ln_1.weight torch.Size([512])
_orig_mod.transformer.h.0.attn.c_attn.weight torch.Size([1536, 512])
_orig_mod.transformer.h.0.attn.c_proj.weight torch.Size([512, 512])
_orig_mod.transformer.h.0.ln_2.weight torch.Size([512])
_orig_mod.transformer.h.0.mlp.c_fc.weight torch.Size([2048, 512])
_orig_mod.transformer.h.0.mlp.c_proj.weight torch.Size([512, 2048])
_orig_mod.transformer.ln_f.weight torch.Size([512])
_orig_mod.lm_head.weight torch.Size([32, 512])
Moving model to device:  cpu


In [7]:
print(model)

HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (pos_embed): PosEmbed()
  (hook_pos_embed): HookPoint()
  (blocks): ModuleList(
    (0-7): 8 x TransformerBlock(
      (ln1): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (ln2): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
      )
      (mlp): MLP(
        (hook_pre): HookPoint()
        (hook_post): HookPoint()
      )
      (hook_attn_in): HookPoint()
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): HookPoint()
      (hook_mlp_in): HookPoint()
      (hook_attn_out): HookPoint()
      (hook_mlp_out): HookPoint()
      (hoo

In [8]:
model.load_and_process_state_dict(convert_nanogpt_weights(synthetic_checkpoint, cfg))
recorded_model_name = model_name.split(".")[0]
torch.save(model.state_dict(), f"{MODEL_DIR}tf_lens_{recorded_model_name}.pth")

In [5]:
print(model)

HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (pos_embed): PosEmbed()
  (hook_pos_embed): HookPoint()
  (blocks): ModuleList(
    (0-7): 8 x TransformerBlock(
      (ln1): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (ln2): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
      )
      (mlp): MLP(
        (hook_pre): HookPoint()
        (hook_post): HookPoint()
      )
      (hook_attn_in): HookPoint()
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): HookPoint()
      (hook_mlp_in): HookPoint()
      (hook_attn_out): HookPoint()
      (hook_mlp_out): HookPoint()
      (hoo

In [4]:
# An example input
sample_input = torch.tensor([[15, 6, 4, 27, 9, 0, 25, 10, 0, 7, 4, 19]]).to(device)
# sample_input = torch.tensor([[15, 6, 4, 27, 9]])
# The argmax of the output (ie the most likely next move from each position)
sample_output = torch.tensor([[6, 4, 27, 9, 0, 27, 10, 0, 7, 4, 19, 28]])
output1 = model(sample_input)
print(output1.size())
model_output = output1.argmax(dim=-1)

print(model_output)
print(sample_output == model_output)

# For this particular sample_input, any model with decent chess skill should output sample_output.
# So, this assert will definitely fail for a randomly initialized model, and may fail for models with low skill.
# But, I've never seen that happen, so I'm keeping it simple for now. For a more robust test, use the nanogpt_to_transformer_lens.ipynb notebook.
# This notebook actually runs the sample input through the original nanogpt model, and then through the converted transformer lens model.
assert torch.all(sample_output == model_output)

torch.Size([1, 12, 32])
tensor([[ 6,  4, 27,  9,  0, 27, 10,  0,  7,  4, 19, 28]])
tensor([[True, True, True, True, True, True, True, True, True, True, True, True]])
